In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add src to path
sys.path.append('../src')

from pipeline import (
    apply_roi_mask, 
    apply_color_threshold, 
    apply_canny,
    detect_lines_hough, 
    filter_lines_by_slope, 
    fit_lane_line,
    extrapolate_line,
    draw_lines
)
from lane_change import LaneChangeDetector

print("✅ Imports successful!")

## 📹 Load Video and Sample Frames Every 5 Seconds

In [ ]:
# Video path
video_path = '../data/processed/highway_clip.mp4'

# Open video and get properties
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap.release()

duration = total_frames / fps

print(f"📊 Video Info:")
print(f"   Total frames: {total_frames}")
print(f"   FPS: {fps}")
print(f"   Duration: {duration:.1f} seconds ({duration/60:.1f} minutes)")
print(f"   Resolution: {width}x{height}")

# Sample every 5 seconds
sample_interval = 5  # seconds
frames_per_sample = fps * sample_interval
frame_indices = list(range(0, total_frames, frames_per_sample))

print(f"\n🎯 Sampling Strategy:")
print(f"   Sample every {sample_interval} seconds")
print(f"   Total samples: {len(frame_indices)}")
print(f"   Frame indices: {frame_indices[:5]}...{frame_indices[-5:]}")

## 🔬 Process All Sampled Frames with Full Pipeline

Store all intermediate results for detailed analysis

In [ ]:
# Storage for all pipeline stages
frames_data = []

cap = cv2.VideoCapture(video_path)

for frame_idx in frame_indices:
    # Jump to specific frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    ret, frame = cap.read()
    
    if not ret:
        print(f"⚠️ Failed to read frame {frame_idx}")
        continue
    
    # Calculate time for camera angle adjustment
    time_sec = frame_idx / fps
    
    # Calculate dynamic ROI based on video position
    progress = frame_idx / total_frames  # 0.0 to 1.0
    
# ...existing code...

    # 3-PHASE ROI: Normal -> Uphill (sky issue) -> Close view
    if time_sec < 252:
        # PHASE 1: Normal driving (0-252s) - CONSTANT 0.62
        dynamic_top_y = 0.62
        
        top_left_x = 0.40
        top_right_x = 0.65
        
    elif 252 <= time_sec < 261:
        # PHASE 2: UPHILL (252-261s) - Camera sees SKY, ROI at 0.70 ⛰️
        dynamic_top_y = 0.70
        
        top_left_x = 0.40
        top_right_x = 0.65
        
    else:
        # PHASE 3: After uphill (261s+) - Camera drops, ROI at 0.60
        dynamic_top_y = 0.60
        
        top_left_x = 0.40
        top_right_x = 0.65
    
    # ============ PIPELINE ============
    # 1. ROI with dynamic positioning and shape
    frame_roi, roi_pts = apply_roi_mask(
        frame, 
        top_y_ratio=dynamic_top_y,
        top_left_x_ratio=top_left_x,
        top_right_x_ratio=top_right_x
    )
    
    # 2. Color threshold
    color_mask = apply_color_threshold(frame_roi)
    
    # 3. Canny edges
    edges = apply_canny(color_mask)
    
    # 4. Detect lines - current threshold=15
    lines = detect_lines_hough(edges, threshold=15, min_line_length=40, max_line_gap=20)
    
    # 5. Filter by slope - CORRECTED to use existing parameters
    left_lines, right_lines = filter_lines_by_slope(
        lines, 
        min_slope=0.5
    )
    
    # 6. Fit lane lines
    left_lane = fit_lane_line(left_lines, height)
    right_lane = fit_lane_line(right_lines, height)
    
    # Create visualization of detected line segments
    lines_visual = frame.copy()
    if left_lines:
        for line in left_lines:
            x1, y1, x2, y2 = line
            cv2.line(lines_visual, (x1, y1), (x2, y2), (255, 0, 255), 2)  # Magenta
    if right_lines:
        for line in right_lines:
            x1, y1, x2, y2 = line
            cv2.line(lines_visual, (x1, y1), (x2, y2), (0, 255, 255), 2)  # Cyan
    
    # ROI visualization
    frame_with_roi = frame.copy()
    cv2.polylines(frame_with_roi, roi_pts, isClosed=True, color=(0, 255, 0), thickness=3)
    
    # Store all data
    frames_data.append({
        'frame_idx': frame_idx,
        'time': time_sec,
        'progress': progress,
        'roi_top_y': dynamic_top_y,
        'frame': frame.copy(),
        'frame_with_roi': frame_with_roi,
        'frame_roi': frame_roi.copy(),
        'color_mask': color_mask.copy(),
        'edges': edges.copy(),
        'lines_visual': lines_visual,
        'left_lane': left_lane,
        'right_lane': right_lane,
        'num_left_lines': len(left_lines) if left_lines else 0,
        'num_right_lines': len(right_lines) if right_lines else 0,
        'roi_pts': roi_pts
    })
    
    if len(frames_data) % 10 == 0:
        print(f"Processed {len(frames_data)} frames...")

cap.release()

print(f"\n✅ Processed {len(frames_data)} frames total")
print(f"   Time range: {frames_data[0]['time']:.1f}s to {frames_data[-1]['time']:.1f}s")

## 📊 Quick Overview: Detection Success Rate

In [ ]:
# Count successful detections
left_detected = sum(1 for d in frames_data if d['left_lane'] is not None)
right_detected = sum(1 for d in frames_data if d['right_lane'] is not None)
both_detected = sum(1 for d in frames_data if d['left_lane'] is not None and d['right_lane'] is not None)

total = len(frames_data)

print(f"Detection Success Rate:")
print(f"   Left lane:  {left_detected}/{total} ({100*left_detected/total:.1f}%)")
print(f"   Right lane: {right_detected}/{total} ({100*right_detected/total:.1f}%)")
print(f"   Both lanes: {both_detected}/{total} ({100*both_detected/total:.1f}%)")

# Average line segments detected
avg_left = np.mean([d['num_left_lines'] for d in frames_data])
avg_right = np.mean([d['num_right_lines'] for d in frames_data])

print(f"\nAverage Line Segments Detected:")
print(f"   Left:  {avg_left:.1f}")
print(f"   Right: {avg_right:.1f}")

## 🎬 Full Pipeline Visualization

Show all 6 stages for each sampled frame

In [ ]:
# Visualize EVERY sampled frame with full pipeline
for i, data in enumerate(frames_data):
    frame_idx = data['frame_idx']
    time_sec = data['time']
    
    # Create final result
    final_result = data['frame'].copy()
    
    if data['left_lane'] is not None and data['right_lane'] is not None:
        # Extrapolate lines
        roi_bottom_y = data['roi_pts'][0][0][1]
        roi_top_y = data['roi_pts'][0][3][1]
        roi_height = roi_bottom_y - roi_top_y
        shorten_top = int(roi_height * 0.15)
        
        extended_bottom_y = height
        extended_top_y = roi_top_y + shorten_top
        
        left_ext = extrapolate_line(data['left_lane'], extended_bottom_y, extended_top_y)
        right_ext = extrapolate_line(data['right_lane'], extended_bottom_y, extended_top_y)
        
        # Draw polygon
        pts = np.array([
            [left_ext[0], left_ext[1]],
            [left_ext[2], left_ext[3]],
            [right_ext[2], right_ext[3]],
            [right_ext[0], right_ext[1]]
        ], dtype=np.int32)
        
        overlay = final_result.copy()
        cv2.fillPoly(overlay, [pts], (0, 255, 0))
        cv2.addWeighted(overlay, 0.3, final_result, 0.7, 0, final_result)
        
        # Draw lines
        cv2.line(final_result, (left_ext[0], left_ext[1]), (left_ext[2], left_ext[3]), (255, 0, 255), 4)
        cv2.line(final_result, (right_ext[0], right_ext[1]), (right_ext[2], right_ext[3]), (0, 255, 255), 4)
    
    # ============ DISPLAY ============
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Row 1
    # 1. Original with ROI
    axes[0, 0].imshow(cv2.cvtColor(data['frame_with_roi'], cv2.COLOR_BGR2RGB))
    axes[0, 0].set_title(f'1. Original + ROI\nROI top_y = {data["roi_top_y"]:.2f}', fontsize=11, fontweight='bold')
    axes[0, 0].axis('off')
    
    # 2. ROI Applied
    axes[0, 1].imshow(cv2.cvtColor(data['frame_roi'], cv2.COLOR_BGR2RGB))
    axes[0, 1].set_title('2. After ROI Masking', fontsize=11, fontweight='bold')
    axes[0, 1].axis('off')
    
    # 3. Color Threshold
    axes[0, 2].imshow(data['color_mask'], cmap='gray')
    axes[0, 2].set_title('3. Color Threshold\n(White + Yellow)', fontsize=11, fontweight='bold')
    axes[0, 2].axis('off')
    
    # Row 2
    # 4. Canny Edges
    axes[1, 0].imshow(data['edges'], cmap='gray')
    axes[1, 0].set_title('4. Canny Edge Detection', fontsize=11, fontweight='bold')
    axes[1, 0].axis('off')
    
    # 5. Detected Line Segments
    axes[1, 1].imshow(cv2.cvtColor(data['lines_visual'], cv2.COLOR_BGR2RGB))
    title = f'5. Hough Lines (threshold=15)\n'
    title += f'Left: {data["num_left_lines"]} | Right: {data["num_right_lines"]}'
    axes[1, 1].set_title(title, fontsize=11, fontweight='bold')
    axes[1, 1].axis('off')
    
    # 6. Final Result
    axes[1, 2].imshow(cv2.cvtColor(final_result, cv2.COLOR_BGR2RGB))
    success = '✅ SUCCESS' if (data['left_lane'] and data['right_lane']) else '❌ FAILED'
    axes[1, 2].set_title(f'6. Final Result\n{success}', fontsize=11, fontweight='bold',
                         color='green' if data['left_lane'] and data['right_lane'] else 'red')
    axes[1, 2].axis('off')
    
    # Main title
    fig.suptitle(f'Frame {frame_idx} | Time: {time_sec:.1f}s | Progress: {data["progress"]*100:.1f}%', 
                 fontsize=16, fontweight='bold', y=0.98)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    print(f"\n{'='*80}")
    print(f"Frame {frame_idx} | Time: {time_sec:.1f}s ({int(time_sec//60)}:{int(time_sec%60):02d})")
    print(f"ROI top_y ratio: {data['roi_top_y']:.3f}")
    print(f"Left segments: {data['num_left_lines']} | Right segments: {data['num_right_lines']}")
    print(f"Left lane fitted: {'✅' if data['left_lane'] else '❌'}")
    print(f"Right lane fitted: {'✅' if data['right_lane'] else '❌'}")
    if data['left_lane']:
        print(f"Left lane params: m={data['left_lane'][0]:.4f}, b={data['left_lane'][1]:.1f}")
    if data['right_lane']:
        print(f"Right lane params: m={data['right_lane'][0]:.4f}, b={data['right_lane'][1]:.1f}")
    print(f"{'='*80}\n")

## 🔍 Problem Analysis

Identify specific issues across the video

In [ ]:
# Find frames with issues
print("🚨 Frames with ZERO line segments detected:\n")
for d in frames_data:
    if d['num_left_lines'] == 0 or d['num_right_lines'] == 0:
        print(f"   Frame {d['frame_idx']} ({d['time']:.1f}s): Left={d['num_left_lines']}, Right={d['num_right_lines']}")

print("\n🚨 Frames with failed lane fitting:\n")
for d in frames_data:
    if d['left_lane'] is None or d['right_lane'] is None:
        print(f"   Frame {d['frame_idx']} ({d['time']:.1f}s): Left={'✅' if d['left_lane'] else '❌'}, Right={'✅' if d['right_lane'] else '❌'}")

print("\n🔎 Frames with very few line segments (< 5):\n")
for d in frames_data:
    if 0 < d['num_left_lines'] < 5 or 0 < d['num_right_lines'] < 5:
        print(f"   Frame {d['frame_idx']} ({d['time']:.1f}s): Left={d['num_left_lines']}, Right={d['num_right_lines']}")

## 📈 Visualize Detection Quality Over Time

In [ ]:
# Plot line segment counts over time
times = [d['time'] for d in frames_data]
left_counts = [d['num_left_lines'] for d in frames_data]
right_counts = [d['num_right_lines'] for d in frames_data]

fig, axes = plt.subplots(3, 1, figsize=(16, 10))

# Line segment counts
axes[0].plot(times, left_counts, 'o-', color='magenta', label='Left', markersize=4, linewidth=2)
axes[0].plot(times, right_counts, 'o-', color='cyan', label='Right', markersize=4, linewidth=2)
axes[0].set_ylabel('Line Segments', fontsize=12, fontweight='bold')
axes[0].set_title('Hough Line Detection Over Time', fontsize=14, fontweight='bold')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(5, color='red', linestyle='--', alpha=0.5, label='Min threshold')

# ROI position over time
roi_tops = [d['roi_top_y'] for d in frames_data]
axes[1].plot(times, roi_tops, 'o-', color='green', markersize=4, linewidth=2)
axes[1].set_ylabel('ROI top_y ratio', fontsize=12, fontweight='bold')
axes[1].set_title('Dynamic ROI Position Over Time', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].invert_yaxis()  # Lower values = looking further

# Detection success
success = [1 if (d['left_lane'] and d['right_lane']) else 0 for d in frames_data]
axes[2].fill_between(times, 0, success, color='green', alpha=0.5, label='Both lanes detected')
axes[2].fill_between(times, success, [1]*len(times), color='red', alpha=0.3, label='Detection failed')
axes[2].set_xlabel('Time (seconds)', fontsize=12, fontweight='bold')
axes[2].set_ylabel('Detection', fontsize=12, fontweight='bold')
axes[2].set_title('Lane Detection Success Over Time', fontsize=14, fontweight='bold')
axes[2].set_yticks([0, 1])
axes[2].set_yticklabels(['Failed', 'Success'])
axes[2].legend(loc='upper right')
axes[2].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## 📐 Slope Analysis: Before and After 255 seconds

Analyze the slope distribution to set better thresholds for each part of the video

In [ ]:
# Separate frames into two groups based on camera angle change at 255s
frames_before_255 = [d for d in frames_data if d['time'] < 255]
frames_after_255 = [d for d in frames_data if d['time'] >= 255]

print(f"Frames before 255s: {len(frames_before_255)}")
print(f"Frames after 255s: {len(frames_after_255)}\n")

# Function to calculate slopes from line segments
def get_slopes_from_lines(lines):
    """Calculate slopes from line segments"""
    slopes = []
    if lines:
        for line in lines:
            x1, y1, x2, y2 = line
            if x2 - x1 != 0:  # Avoid division by zero
                slope = (y2 - y1) / (x2 - x1)
                slopes.append(slope)
    return slopes

# Collect all slopes for each period
left_slopes_before = []
right_slopes_before = []
left_slopes_after = []
right_slopes_after = []

# Process each frame and collect slopes
cap = cv2.VideoCapture(video_path)

for data in frames_data:
    frame_idx = data['frame_idx']
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    ret, frame = cap.read()
    
    if not ret:
        continue
    
    # Re-run pipeline to get raw lines before filtering
    frame_roi, _ = apply_roi_mask(frame, top_y_ratio=data['roi_top_y'])
    color_mask = apply_color_threshold(frame_roi)
    edges = apply_canny(color_mask)
    lines = detect_lines_hough(edges, threshold=15, min_line_length=40, max_line_gap=20)
    
    if lines is None:
        continue
    
    # Calculate slopes for all lines
    for line in lines:
        x1, y1, x2, y2 = line[0]
        if x2 - x1 == 0:
            continue
        slope = (y2 - y1) / (x2 - x1)
        
        # Separate left (negative) and right (positive) slopes
        if data['time'] < 255:
            if slope < 0:
                left_slopes_before.append(slope)
            else:
                right_slopes_before.append(slope)
        else:
            if slope < 0:
                left_slopes_after.append(slope)
            else:
                right_slopes_after.append(slope)

cap.release()

# Print statistics
print("="*80)
print("SLOPE STATISTICS - BEFORE 255 SECONDS (Camera looking far)")
print("="*80)
print(f"\nLeft lane (negative slopes):")
if left_slopes_before:
    print(f"  Count: {len(left_slopes_before)}")
    print(f"  Mean: {np.mean(left_slopes_before):.3f}")
    print(f"  Median: {np.median(left_slopes_before):.3f}")
    print(f"  Std Dev: {np.std(left_slopes_before):.3f}")
    print(f"  Range: [{np.min(left_slopes_before):.3f}, {np.max(left_slopes_before):.3f}]")
    print(f"  10th percentile: {np.percentile(left_slopes_before, 10):.3f}")
    print(f"  90th percentile: {np.percentile(left_slopes_before, 90):.3f}")

print(f"\nRight lane (positive slopes):")
if right_slopes_before:
    print(f"  Count: {len(right_slopes_before)}")
    print(f"  Mean: {np.mean(right_slopes_before):.3f}")
    print(f"  Median: {np.median(right_slopes_before):.3f}")
    print(f"  Std Dev: {np.std(right_slopes_before):.3f}")
    print(f"  Range: [{np.min(right_slopes_before):.3f}, {np.max(right_slopes_before):.3f}]")
    print(f"  10th percentile: {np.percentile(right_slopes_before, 10):.3f}")
    print(f"  90th percentile: {np.percentile(right_slopes_before, 90):.3f}")

print("\n" + "="*80)
print("SLOPE STATISTICS - AFTER 255 SECONDS (Camera drops lower)")
print("="*80)
print(f"\nLeft lane (negative slopes):")
if left_slopes_after:
    print(f"  Count: {len(left_slopes_after)}")
    print(f"  Mean: {np.mean(left_slopes_after):.3f}")
    print(f"  Median: {np.median(left_slopes_after):.3f}")
    print(f"  Std Dev: {np.std(left_slopes_after):.3f}")
    print(f"  Range: [{np.min(left_slopes_after):.3f}, {np.max(left_slopes_after):.3f}]")
    print(f"  10th percentile: {np.percentile(left_slopes_after, 10):.3f}")
    print(f"  90th percentile: {np.percentile(left_slopes_after, 90):.3f}")

print(f"\nRight lane (positive slopes):")
if right_slopes_after:
    print(f"  Count: {len(right_slopes_after)}")
    print(f"  Mean: {np.mean(right_slopes_after):.3f}")
    print(f"  Median: {np.median(right_slopes_after):.3f}")
    print(f"  Std Dev: {np.std(right_slopes_after):.3f}")
    print(f"  Range: [{np.min(right_slopes_after):.3f}, {np.max(right_slopes_after):.3f}]")
    print(f"  10th percentile: {np.percentile(right_slopes_after, 10):.3f}")
    print(f"  90th percentile: {np.percentile(right_slopes_after, 90):.3f}")

# Visualize slope distributions
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Before 255s - Left lanes
axes[0, 0].hist(left_slopes_before, bins=50, color='magenta', alpha=0.7, edgecolor='black')
axes[0, 0].axvline(np.median(left_slopes_before), color='red', linestyle='--', linewidth=2, label=f'Median: {np.median(left_slopes_before):.3f}')
axes[0, 0].axvline(-0.5, color='yellow', linestyle='--', linewidth=2, label='Current min_slope: -0.5')
axes[0, 0].set_title('BEFORE 255s - Left Lane Slopes', fontsize=13, fontweight='bold')
axes[0, 0].set_xlabel('Slope')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Before 255s - Right lanes
axes[0, 1].hist(right_slopes_before, bins=50, color='cyan', alpha=0.7, edgecolor='black')
axes[0, 1].axvline(np.median(right_slopes_before), color='red', linestyle='--', linewidth=2, label=f'Median: {np.median(right_slopes_before):.3f}')
axes[0, 1].axvline(0.5, color='yellow', linestyle='--', linewidth=2, label='Current min_slope: 0.5')
axes[0, 1].set_title('BEFORE 255s - Right Lane Slopes', fontsize=13, fontweight='bold')
axes[0, 1].set_xlabel('Slope')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# After 255s - Left lanes
if left_slopes_after:
    axes[1, 0].hist(left_slopes_after, bins=50, color='magenta', alpha=0.7, edgecolor='black')
    axes[1, 0].axvline(np.median(left_slopes_after), color='red', linestyle='--', linewidth=2, label=f'Median: {np.median(left_slopes_after):.3f}')
    axes[1, 0].axvline(-0.5, color='yellow', linestyle='--', linewidth=2, label='Current min_slope: -0.5')
axes[1, 0].set_title('AFTER 255s - Left Lane Slopes', fontsize=13, fontweight='bold')
axes[1, 0].set_xlabel('Slope')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# After 255s - Right lanes
if right_slopes_after:
    axes[1, 1].hist(right_slopes_after, bins=50, color='cyan', alpha=0.7, edgecolor='black')
    axes[1, 1].axvline(np.median(right_slopes_after), color='red', linestyle='--', linewidth=2, label=f'Median: {np.median(right_slopes_after):.3f}')
    axes[1, 1].axvline(0.5, color='yellow', linestyle='--', linewidth=2, label='Current min_slope: 0.5')
axes[1, 1].set_title('AFTER 255s - Right Lane Slopes', fontsize=13, fontweight='bold')
axes[1, 1].set_xlabel('Slope')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("💡 RECOMMENDATIONS")
print("="*80)
print("\nBased on the slope analysis, consider:")
print("\n1. BEFORE 255s (camera far):")
if left_slopes_before:
    suggested_left = abs(np.percentile(left_slopes_before, 10))
    print(f"   Left min_slope: {suggested_left:.2f} (currently -0.5)")
if right_slopes_before:
    suggested_right = np.percentile(right_slopes_before, 10)
    print(f"   Right min_slope: {suggested_right:.2f} (currently 0.5)")

print("\n2. AFTER 255s (camera lower):")
if left_slopes_after:
    suggested_left = abs(np.percentile(left_slopes_after, 10))
    print(f"   Left min_slope: {suggested_left:.2f}")
if right_slopes_after:
    suggested_right = np.percentile(right_slopes_after, 10)
    print(f"   Right min_slope: {suggested_right:.2f}")

print("\n3. Consider using adaptive min_slope based on time!")
print("="*80)

## 📊 Final Slope Recommendations Summary

Based on the analysis, here's what we should use for adaptive min_slope

In [ ]:
print("="*80)
print("🎯 FINAL RECOMMENDATIONS FOR ADAPTIVE MIN_SLOPE")
print("="*80)

if left_slopes_before and right_slopes_before:
    # Before 255s - use 10th percentile as threshold (filters out 10% outliers)
    left_min_before = abs(np.percentile(left_slopes_before, 10))
    right_min_before = np.percentile(right_slopes_before, 10)
    
    print("\n📍 BEFORE 255 SECONDS (Camera far, looking ahead):")
    print(f"   Left lane:  min_slope = {left_min_before:.2f}")
    print(f"   Right lane: min_slope = {right_min_before:.2f}")
    print(f"   Current setting: 0.5 (same for both)")
    print(f"   → Change needed: {'YES ✅' if abs(left_min_before - 0.5) > 0.1 or abs(right_min_before - 0.5) > 0.1 else 'NO ❌'}")

if left_slopes_after and right_slopes_after:
    # After 255s - use 10th percentile
    left_min_after = abs(np.percentile(left_slopes_after, 10))
    right_min_after = np.percentile(right_slopes_after, 10)
    
    print("\n📍 AFTER 255 SECONDS (Camera lower, closer view):")
    print(f"   Left lane:  min_slope = {left_min_after:.2f}")
    print(f"   Right lane: min_slope = {right_min_after:.2f}")
    print(f"   Current setting: 0.5 (same for both)")
    print(f"   → Change needed: {'YES ✅' if abs(left_min_after - 0.5) > 0.1 or abs(right_min_after - 0.5) > 0.1 else 'NO ❌'}")

print("\n" + "="*80)
print("💡 IMPLEMENTATION STRATEGY")
print("="*80)

if left_slopes_before and right_slopes_before and left_slopes_after and right_slopes_after:
    # Check if slopes differ significantly between periods
    left_diff = abs(abs(np.percentile(left_slopes_before, 10)) - abs(np.percentile(left_slopes_after, 10)))
    right_diff = abs(np.percentile(right_slopes_before, 10) - np.percentile(right_slopes_after, 10))
    
    if left_diff > 0.1 or right_diff > 0.1:
        print("\n✅ ADAPTIVE THRESHOLDS RECOMMENDED")
        print("   The slopes differ significantly between video sections.")
        print("   Use time-based adaptive min_slope in your pipeline:")
        print("\n   if time_sec < 255:")
        print(f"       left_min_slope = {left_min_before:.2f}")
        print(f"       right_min_slope = {right_min_before:.2f}")
        print("   else:")
        print(f"       left_min_slope = {left_min_after:.2f}")
        print(f"       right_min_slope = {right_min_after:.2f}")
    else:
        print("\n❌ ADAPTIVE THRESHOLDS NOT NEEDED")
        print("   The slopes are similar throughout the video.")
        avg_left = (abs(np.percentile(left_slopes_before, 10)) + abs(np.percentile(left_slopes_after, 10))) / 2
        avg_right = (np.percentile(right_slopes_before, 10) + np.percentile(right_slopes_after, 10)) / 2
        print(f"   Use constant min_slope:")
        print(f"       left_min_slope = {avg_left:.2f}")
        print(f"       right_min_slope = {avg_right:.2f}")

print("\n" + "="*80)

## 💡 Findings and Recommendations

**Based on the analysis above, check for:**

1. **ROI Issues:**
   - Is the ROI too low at the start? (top_y > 0.65)
   - Does it cut off important lane markers?
   - Should we adjust the dynamic range?

2. **Line Detection Issues:**
   - Are there specific time ranges with poor detection?
   - Is threshold=15 too high? (try lower like 10)
   - Are min_line_length=40 and max_line_gap=20 appropriate?

3. **Color Thresholding:**
   - Does the color mask miss lane markers in certain frames?
   - Check if lighting conditions affect detection

**Next Steps:**
- Adjust parameters based on problematic frames identified above
- Test with different threshold values
- Consider adaptive ROI based on detection quality